<a href="https://colab.research.google.com/github/OmdenaAI/omdena-colombia-socialnetwork/blob/task-5-visualization/src/tasks/task-5-visualization/Carlos_extract_colombian_locations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
def clean_text(text):
  text = text.lower()
  text = text.replace("á","a")
  text = text.replace("à", "a")
  text = text.replace("é","e")
  text = text.replace("í","i")
  text = text.replace("ó","o")
  text = text.replace("ú","u")
  text = text.replace("ü","u")
  text = text.strip()
  text = text.replace("🇨🇴","")
  text = text.replace(",d.c","")
  text = text.replace("d.c","")
  text = text.replace(", d.c","")
  text = text.replace("."," ")
  text = text.replace("&","")
  text = text.replace("#","")
  text = text.replace("-",",")
  text = text.replace("/",",")
  #text = text.replace("polombia","colombia")
  #text = text.replace("colombia","")
  text = text.replace(" dc","")
  text = text.replace("@","")
  #text = text.replace("¬", " ")
  return text

In [3]:
df = pd.read_csv("User_Location Dictionary.csv", index_col="Unnamed: 0")
df.dropna(how="all", subset=["Location"],inplace=True)
df.head()

,User,Location
2,DS_NAN,BS AS ARGENTINA
3,Robertob_ad,venezuela
5,julio2j,Venezuela
8,07mike,&
10,josealejandrou,"Maracaibo, Venezuela"


In [4]:
df.shape

(6559, 2)

In [5]:
df["cleaned_location"] = df["Location"].apply(clean_text)
df = df[df["cleaned_location"] != ""]
df.head()

,User,Location,cleaned_location
2,DS_NAN,BS AS ARGENTINA,bs as argentina
3,Robertob_ad,venezuela,venezuela
5,julio2j,Venezuela,venezuela
10,josealejandrou,"Maracaibo, Venezuela","maracaibo, venezuela"
12,magsegovia,"Lima, Peru","lima, peru"


In [6]:
df.shape

(6550, 3)

In [7]:
def is_not_colombian(cleaned_location):
  countries = ["argentina", "venezuela", "peru", "chile", "spain", 
               "españa", "france", "costa rica", "uruguay", "cuba",
               "italia", "england", "ecuador", "el salvador", "nicaragua",
               "italy", "mexico", "rusia"]
  ans = False 
  for country in countries: 
    if country in cleaned_location: 
      ans = True
    else: 
      ans = ans or False
  return ans  

In [8]:
df["not_colombian"] = df["cleaned_location"].apply(is_not_colombian)
df.head()

,User,Location,cleaned_location,not_colombian
2,DS_NAN,BS AS ARGENTINA,bs as argentina,True
3,Robertob_ad,venezuela,venezuela,True
5,julio2j,Venezuela,venezuela,True
10,josealejandrou,"Maracaibo, Venezuela","maracaibo, venezuela",True
12,magsegovia,"Lima, Peru","lima, peru",True


In [9]:
df = df[df["not_colombian"] == False]

In [10]:
df.shape

(5867, 4)

In [11]:
colombian_locations = pd.read_csv("table-1.csv",usecols=["City", "Department"])
colombian_locations["city_preprocessed"] = colombian_locations["City"].apply(clean_text)
colombian_locations["department_preprocessed"] = colombian_locations["Department"].apply(clean_text)

colombian_dict_cities = {}
colombian_dict_departments = {}

for _, row in colombian_locations.iterrows():
  colombian_dict_cities[row["city_preprocessed"]] = row["department_preprocessed"]
  colombian_dict_departments[row["department_preprocessed"]] = row["city_preprocessed"]

colombian_dict_cities["santiago de cali"] = colombian_dict_cities["cali"]
colombian_locations.head()

,City,Department,city_preprocessed,department_preprocessed
0,Bogotá,Capital District,bogota,capital district
1,Leticia,Amazonas,leticia,amazonas
2,El Encanto,Amazonas,el encanto,amazonas
3,La Chorrera,Amazonas,la chorrera,amazonas
4,La Pedrera,Amazonas,la pedrera,amazonas


In [12]:
df["location_array"] = df["cleaned_location"].str.split(",")
df.head()

,User,Location,cleaned_location,not_colombian,location_array
15,Cephalosporium,"Cali, Colombia","cali, colombia",False,"[cali, colombia]"
16,Lucio_Biglione,"#General Cabrera,Cordoba","general cabrera,cordoba",False,"[general cabrera, cordoba]"
21,FTG84,Corvera d'Asturies,corvera d'asturies,False,[corvera d'asturies]
27,arielucho1970,Ciudad Autónoma de Buenos Aire,ciudad autonoma de buenos aire,False,[ciudad autonoma de buenos aire]
28,Camiloramirez_s,"Bogotá , Colombia","bogota , colombia",False,"[bogota , colombia]"


In [13]:
def get_complete_location_from_csv(location_array):
  city = ""
  department = ""
  country = ""
  for location_item in location_array: 
    location_item = location_item.strip()
    location_item = location_item.replace(" colombia", "")
    location_item = location_item.replace("colombia ", "")
    if location_item == "colombia" and len(location_array) == 1:
      city = "bogota"
      department = "capital district"
      country = "colombia"
    elif colombian_dict_cities.get(location_item):
      city = location_item
      department = colombian_dict_cities.get(location_item)
      country = "colombia"
      break
    elif colombian_dict_departments.get(location_item):
      city = colombian_dict_departments.get(location_item)
      department = location_item 
      country = "colombia"
  return pd.Series([city,department,country])

In [14]:
df[["city", "department", "country"]] = df["location_array"].apply(get_complete_location_from_csv)

In [31]:
df_co = df[df["country"] == "colombia"]
df_co

,User,Location,cleaned_location,not_colombian,location_array,city,department,country,address,latitude,longitude
15,Cephalosporium,"Cali, Colombia","cali, colombia",False,"[cali, colombia]",cali,valle del cauca,colombia,NaN,NaN,NaN
16,Lucio_Biglione,"#General Cabrera,Cordoba","general cabrera,cordoba",False,"[general cabrera, cordoba]",cordoba,quindio,colombia,NaN,NaN,NaN
28,Camiloramirez_s,"Bogotá , Colombia","bogota , colombia",False,"[bogota , colombia]",bogota,capital district,colombia,NaN,NaN,NaN
33,RichardRiCampo,Cali - Valle,"cali , valle",False,"[cali , valle]",cali,valle del cauca,colombia,NaN,NaN,NaN
38,activistica,Colombia,colombia,False,[colombia],bogota,capital district,colombia,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10535,G_abrielaaa_,"Bogotá, D.C., Colombia","bogota, , colombia",False,"[bogota, , colombia]",bogota,capital district,colombia,NaN,NaN,NaN
10536,jennypaosuarezg,BOGOTA - COLOMBIA,"bogota , colombia",False,"[bogota , colombia]",bogota,capital district,colombia,NaN,NaN,NaN
10537,Martinpz02,"Bogotá, D.C., Colombia","bogota, , colombia",False,"[bogota, , colombia]",bogota,capital district,colombia,NaN,NaN,NaN
10539,somosmestizzo,bogota,bogota,False,[bogota],bogota,capital district,colombia,NaN,NaN,NaN


In [32]:
df[df["country"] != "colombia"]

,User,Location,cleaned_location,not_colombian,location_array,city,department,country,address,latitude,longitude
21,FTG84,Corvera d'Asturies,corvera d'asturies,False,[corvera d'asturies],,,,NaN,NaN,NaN
27,arielucho1970,Ciudad Autónoma de Buenos Aire,ciudad autonoma de buenos aire,False,[ciudad autonoma de buenos aire],,,,NaN,NaN,NaN
35,ConvocatoriasGz,Galiza,galiza,False,[galiza],,,,NaN,NaN,NaN
37,williamgarzon86,http://williamgarzon86,"http:,,williamgarzon86",False,"[http:, , williamgarzon86]",,,,NaN,NaN,NaN
40,tecnochamana,Theremín.,theremin,False,[theremin ],,,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10435,AndresParbe,Deportivo Cali ois.,deportivo cali ois,False,[deportivo cali ois ],,,,NaN,NaN,NaN
10471,JotaAEmeA,Κολομβία,κολομβία,False,[κολομβία],,,,NaN,NaN,NaN
10477,fevelasquezd,narcoestado asesino colombiano,narcoestado asesino colombiano,False,[narcoestado asesino colombiano],,,,NaN,NaN,NaN
10484,emmanuelrangel6,Bogota Dc. Colombia,bogota colombia,False,[bogota colombia],,,,NaN,NaN,NaN


In [36]:
def get_lat_lon(row):
  geolocator = Nominatim(user_agent="colombia_strike")
  location_str = f"{row['city']}, {row['department']}"
  location = geolocator.geocode(location_str)
  address = location.address if location else ""
  latitude = location.latitude if location else ""
  longitude = location.longitude if location else ""
  if not location:
    print(location_str)
  return pd.Series([address, latitude, longitude])

df_co[["address", "latitude", "longitude"]] = df_co.apply(get_lat_lon, axis = 1)

salazar de las palmas, norte de santander


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [41]:
df_co["address"].isna().sum()

0

In [42]:
df_co["address"].shape

(3851,)

In [45]:
df_co.head()

,User,Location,cleaned_location,not_colombian,location_array,city,department,country,address,latitude,longitude
15,Cephalosporium,"Cali, Colombia","cali, colombia",False,"[cali, colombia]",cali,valle del cauca,colombia,"Cali, Sur, Valle del Cauca, Pacífica, Colombia",3.45179,-76.5325
16,Lucio_Biglione,"#General Cabrera,Cordoba","general cabrera,cordoba",False,"[general cabrera, cordoba]",cordoba,quindio,colombia,"Córdoba, Cordillerana, Quindío, Región Andina,...",4.3956,-75.6518
28,Camiloramirez_s,"Bogotá , Colombia","bogota , colombia",False,"[bogota , colombia]",bogota,capital district,colombia,"Bogotá, Bogotá Distrito Capital, Región Andina...",4.65333,-74.0837
33,RichardRiCampo,Cali - Valle,"cali , valle",False,"[cali , valle]",cali,valle del cauca,colombia,"Cali, Sur, Valle del Cauca, Pacífica, Colombia",3.45179,-76.5325
38,activistica,Colombia,colombia,False,[colombia],bogota,capital district,colombia,"Bogotá, Bogotá Distrito Capital, Región Andina...",4.65333,-74.0837


In [46]:
df_co.to_csv("cleaned_colombian_locations.csv")